## LoRA를 이용한 Stable Diffusion 이미지 생성


In [1]:
# 해당 git을 사전에 설치해야 사용가능
# pip install git+https://github.com/cloneofsimo/lora.git

# 해당 파일을 사전에 설치해야함
'''diffusers>=0.11.0
transformers>=4.25.1
scipy
ftfy
fire
wandb
safetensors
opencv-python
torchvision
mediapipe'''



'diffusers>=0.11.0\ntransformers>=4.25.1\nscipy\nftfy\nfire\nwandb\nsafetensors\nopencv-python\ntorchvision\nmediapipe'

In [2]:
from PIL import Image

In [3]:
import torch

In [4]:
import ipywidgets as widgets

In [5]:
from diffusers import StableDiffusionImg2ImgPipeline

In [ ]:
# 입력이미지 설정, 본인의 이미지 파일에 대한 경로를 입력
init_image = Image.open("/Users/yungchannelforyou/Desktop/Lora/KakaoTalk_Photo_2023-04-03-11-36-04.jpeg").convert("RGB").resize((512, 512))

model_id = "runwayml/stable-diffusion-v1-5" # Stable Diffusion모델 설정

In [ ]:
# img2img를 위한 파이프라인 설정
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(
    "cuda"
)

In [ ]:
prompt = "korea woman"  # 프롬프트는 픽셀 캐릭터로 고정
torch.manual_seed(1)        # 난수 고정 , 이때 설정을 제대로 안하면 검은색 화면이 자주 출력될 수 있음
image = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]  # strength와 scale을 적정하게 설정하여 실험

image  # LoRA 파일없이 이미지 생성


In [ ]:
# LoRA파일 생성을 위한 코드
import os

# 환경 변수 설정
os.environ['MODEL_NAME'] = "runwayml/stable-diffusion-v1-5"
os.environ['INSTANCE_DIR'] = "/Users/yungchannelforyou/Desktop/Lora/dataset"
os.environ['OUTPUT_DIR'] = "/Users/yungchannelforyou/Desktop/Lora/g_output"

# LoRA 미세조정 명령어 실행
!lora_pti \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --train_text_encoder \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --scale_lr \
  --learning_rate_unet=1e-4 \
  --learning_rate_text=1e-5 \
  --learning_rate_ti=5e-4 \
  --color_jitter \
  --lr_scheduler="linear" \
  --lr_warmup_steps=0 \
  --placeholder_tokens="<s1>|<s2>" \
  --use_template="style" \
  --save_steps=100 \
  --max_train_steps_ti=1000 \
  --max_train_steps_tuning=1000 \
  --perform_inversion=True \
  --clip_ti_decay \
  --weight_decay_ti=0.000 \
  --weight_decay_lora=0.001 \
  --continue_inversion \
  --continue_inversion_lr=1e-4 \
  --device="cuda:0" \
  --lora_rank=1 

In [ ]:
from lora_diffusion import patch_pipe, tune_lora_scale

# 생성된 LoRA파일 중 final 파일 적용
patch_pipe(pipe, "/Users/yungchannelforyou/Desktop/Lora/g_output/final_lora.safetensors", patch_text=True, patch_unet=True, patch_ti=True)

torch.manual_seed(1)
image = pipe(prompt=prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]

image # LoRA 적용 후 이미지 생성


#
